# 🧱 Surface Reconstruction from Point Clouds (PCD/PLY) — Colab Notebook

This notebook lets you upload a `.pcd` (or `.ply`, `.xyz`) point cloud and reconstruct a watertight surface mesh using **Poisson** or a lighter **Ball-Pivoting** method, all with **Open3D**.

## What you get
- Normal estimation (if missing)
- Optional voxel downsampling
- Poisson reconstruction with density-based trimming
- Ball-Pivoting reconstruction with auto radii
- Mesh cleaning (remove degenerate/duplicated entities)
- Decimation to target triangle count
- Export to `.ply`, `.stl`, `.obj`, and optionally `.glb`
- Offscreen PNG snapshots of the point cloud and meshes

## Quick start
1. Run **Setup** (next cell) to install dependencies.
2. Run **Upload point cloud (.pcd/.ply)** and select your file.
3. Review **Parameters** and tweak if needed.
4. Run **Poisson Reconstruction** or **Ball-Pivoting Reconstruction**.
5. Download exported files (at the end of each section) or find them in `/content`.

> Tip: If your cloud is huge, start with a larger `VOXEL_SIZE` to downsample (e.g., 0.02–0.05 for meter-scale scenes).


In [1]:
#@title 🔧 Setup (install packages)
!pip -q install open3d==0.19.0 trimesh pygltflib ipywidgets
from google.colab import output
output.enable_custom_widget_manager()
print('✅ Installed. If widgets do not render, re-run this cell once.')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
✅ Installed. If widgets do not render, re-run this cell once.


In [2]:
#@title ⬆️ Upload point cloud (.pcd, .ply, .xyz)
from google.colab import files
up = files.upload()
assert len(up) > 0, 'No file uploaded.'
PC_FILE = list(up.keys())[0]
print('Using file:', PC_FILE)

Saving frame_0007_1755004067806936064.pcd to frame_0007_1755004067806936064.pcd
Using file: frame_0007_1755004067806936064.pcd


In [3]:
#@title ⚙️ Parameters (edit and re-run)
VOXEL_SIZE = 0.01  #@param {type:"number"}
NORMAL_RADIUS = 0.03  #@param {type:"number"}
NORMAL_MAX_NN = 30  #@param {type:"integer"}

# Poisson
POISSON_DEPTH = 10  #@param {type:"integer"}
POISSON_SCALE = 1.1  #@param {type:"number"}
DENSITY_TRIM_QUANTILE = 0.02  #@param {type:"number"}

# Ball-Pivot radii scale factors (based on avg NN distance)
BP_RADII_SCALES = [1.5, 2.5, 3.5]  #@param {type:"raw"}

# Mesh decimation target (0 = skip decimation)
TARGET_TRIS = 200_000  #@param {type:"integer"}

# Snapshot size
SNAP_W, SNAP_H = 1280, 960  #@param {type:"raw"}
print('Parameters set.')

Parameters set.


In [4]:
import numpy as np
import open3d as o3d
import os, math
from pathlib import Path

def load_point_cloud(path: str) -> o3d.geometry.PointCloud:
    ext = Path(path).suffix.lower()
    if ext in ['.pcd', '.ply', '.xyz', '.xyzn', '.xyzrgb', '.pts']:
        pcd = o3d.io.read_point_cloud(path)
        if len(pcd.points) == 0:
            raise ValueError('Loaded point cloud is empty. Wrong file or format?')
        return pcd
    else:
        raise ValueError(f'Unsupported extension: {ext}')

def nan_filter_pcd(pcd: o3d.geometry.PointCloud) -> o3d.geometry.PointCloud:
    pts = np.asarray(pcd.points)
    mask = np.isfinite(pts).all(axis=1)
    if pcd.has_colors():
        cols = np.asarray(pcd.colors)[mask]
    else:
        cols = None
    pcd2 = o3d.geometry.PointCloud()
    pcd2.points = o3d.utility.Vector3dVector(pts[mask])
    if cols is not None:
        pcd2.colors = o3d.utility.Vector3dVector(cols)
    return pcd2

def preprocess_pcd(pcd: o3d.geometry.PointCloud,
                   voxel_size: float,
                   normal_radius: float,
                   normal_max_nn: int) -> o3d.geometry.PointCloud:
    pcd = nan_filter_pcd(pcd)
    if voxel_size and voxel_size > 0:
        pcd = pcd.voxel_down_sample(voxel_size)
    if not pcd.has_normals():
        pcd.estimate_normals(
            search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=normal_radius, max_nn=normal_max_nn)
        )
        pcd.orient_normals_consistent_tangent_plane(k=normal_max_nn)
    return pcd

def average_nn_distance(pcd: o3d.geometry.PointCloud, k: int = 8) -> float:
    if len(pcd.points) < k:
        return 0.01
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    dists = []
    pts = np.asarray(pcd.points)
    for i in range(min(len(pts), 5000)):  # sample up to 5k points for speed
        _, idx, _ = pcd_tree.search_knn_vector_3d(pts[i], k)
        if len(idx) == k:
            nn = pts[idx[1:]] - pts[i]
            d = np.linalg.norm(nn, axis=1).mean()
            if np.isfinite(d):
                dists.append(d)
    return float(np.median(dists)) if dists else 0.01

def clean_mesh(mesh: o3d.geometry.TriangleMesh) -> o3d.geometry.TriangleMesh:
    mesh.remove_duplicated_vertices()
    mesh.remove_duplicated_triangles()
    mesh.remove_degenerate_triangles()
    mesh.remove_non_manifold_edges()
    mesh.compute_vertex_normals()
    return mesh

def decimate_if_needed(mesh: o3d.geometry.TriangleMesh, target_tris: int) -> o3d.geometry.TriangleMesh:
    if target_tris and target_tris > 0 and np.asarray(mesh.triangles).shape[0] > target_tris:
        mesh = mesh.simplify_quadric_decimation(target_tris)
        mesh.compute_vertex_normals()
    return mesh

def save_mesh_all_formats(mesh: o3d.geometry.TriangleMesh, base: str):
    outs = []
    for ext in ['ply', 'stl', 'obj']:
        path = f"{base}.{ext}"
        o3d.io.write_triangle_mesh(path, mesh)
        outs.append(path)
    # Try GLB via trimesh
    try:
        import trimesh as tm
        verts = np.asarray(mesh.vertices)
        faces = np.asarray(mesh.triangles)
        tmm = tm.Trimesh(vertices=verts, faces=faces, process=False)
        tmm.export(f"{base}.glb")
        outs.append(f"{base}.glb")
    except Exception as e:
        print('GLB export skipped (install failed or error):', e)
    return outs

def render_snapshot(geom, out_path: str, w: int = 1280, h: int = 960):
    from open3d.visualization import rendering
    renderer = rendering.OffscreenRenderer(w, h)
    scene = renderer.scene
    scene.set_background(np.array([1.0, 1.0, 1.0, 1.0]))
    mat = rendering.MaterialRecord()
    if isinstance(geom, o3d.geometry.PointCloud):
        mat.shader = 'defaultUnlit'
        mat.point_size = 2.0
    else:
        mat.shader = 'defaultLit'
    scene.add_geometry('geom', geom, mat)
    bbox = geom.get_axis_aligned_bounding_box()
    center = bbox.get_center()
    extent = bbox.get_extent()
    diag = np.linalg.norm(extent)
    cam_pos = center + np.array([diag, diag, diag]) * 0.7
    up = np.array([0.0, 0.0, 1.0])
    scene.camera.look_at(center, cam_pos, up)
    img = renderer.render_to_image()
    o3d.io.write_image(out_path, img)
    return out_path


In [5]:
#@title 🌀 Poisson Reconstruction (with density trimming & decimation)
pcd = load_point_cloud(PC_FILE)
print('Loaded points:', len(pcd.points))

pcd = preprocess_pcd(pcd, VOXEL_SIZE, NORMAL_RADIUS, NORMAL_MAX_NN)
print('Preprocessed points:', len(pcd.points))

# snap_pcd = render_snapshot(pcd, 'pointcloud.png', SNAP_W, SNAP_H)
from IPython.display import Image, display
# display(Image(filename=snap_pcd))

mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcd, depth=POISSON_DEPTH, scale=POISSON_SCALE, linear_fit=False
)
densities = np.asarray(densities)
print('Raw Poisson mesh tris:', np.asarray(mesh.triangles).shape[0])

# Trim low-density vertices
if 0.0 < DENSITY_TRIM_QUANTILE < 0.5:
    thr = np.quantile(densities, DENSITY_TRIM_QUANTILE)
    keep = densities >= thr
    mesh = mesh.select_by_index(np.where(keep)[0])
    print(f'Trimmed vertices below quantile {DENSITY_TRIM_QUANTILE:.2f}.')

mesh = clean_mesh(mesh)
mesh = decimate_if_needed(mesh, TARGET_TRIS)
print('Final Poisson mesh tris:', np.asarray(mesh.triangles).shape[0])

# snap_mesh = render_snapshot(mesh, 'mesh_poisson.png', SNAP_W, SNAP_H)
# display(Image(filename=snap_mesh))

base = Path(PC_FILE).with_suffix('').name + '_poisson'
outs = save_mesh_all_formats(mesh, base)
print('Saved:', outs)

Loaded points: 114688
Preprocessed points: 97114
Raw Poisson mesh tris: 478885
Trimmed vertices below quantile 0.02.
Final Poisson mesh tris: 200000
[Open3D WARNING] Write OBJ can not include triangle normals.
Saved: ['frame_0007_1755004067806936064_poisson.ply', 'frame_0007_1755004067806936064_poisson.stl', 'frame_0007_1755004067806936064_poisson.obj', 'frame_0007_1755004067806936064_poisson.glb']


In [6]:
#@title ⚽ Ball-Pivoting Reconstruction (auto radii from NN distance)
pcd2 = load_point_cloud(PC_FILE)
pcd2 = preprocess_pcd(pcd2, VOXEL_SIZE, NORMAL_RADIUS, NORMAL_MAX_NN)

avg_nn = average_nn_distance(pcd2, k=8)
radii = o3d.utility.DoubleVector([float(avg_nn*s) for s in BP_RADII_SCALES])
print('Avg NN distance ~', avg_nn, '→ radii:', list(radii))

bp_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd2, radii)
bp_mesh = clean_mesh(bp_mesh)
bp_mesh = decimate_if_needed(bp_mesh, TARGET_TRIS)
print('Ball-Pivot mesh tris:', np.asarray(bp_mesh.triangles).shape[0])

# snap_bp = render_snapshot(bp_mesh, 'mesh_ballpivot.png', SNAP_W, SNAP_H)
from IPython.display import Image, display
# display(Image(filename=snap_bp))

base = Path(PC_FILE).with_suffix('').name + '_ballpivot'
outs = save_mesh_all_formats(bp_mesh, base)
print('Saved:', outs)

Avg NN distance ~ 0.03678054624065975 → radii: [0.05517081936098963, 0.09195136560164938, 0.12873191184230914]
Ball-Pivot mesh tris: 170891
[Open3D WARNING] Write OBJ can not include triangle normals.
Saved: ['frame_0007_1755004067806936064_ballpivot.ply', 'frame_0007_1755004067806936064_ballpivot.stl', 'frame_0007_1755004067806936064_ballpivot.obj', 'frame_0007_1755004067806936064_ballpivot.glb']


In [7]:
#@title 💾 Download any output
from google.colab import files
to_get = [
    'pointcloud.png', 'mesh_poisson.png', 'mesh_ballpivot.png'
]
base_poisson = Path(PC_FILE).with_suffix('').name + '_poisson'
base_bp = Path(PC_FILE).with_suffix('').name + '_ballpivot'
for ext in ['ply','stl','obj','glb']:
    to_get.append(f'{base_poisson}.{ext}')
    to_get.append(f'{base_bp}.{ext}')
existing = [p for p in to_get if os.path.exists(p)]
print('Files available:', existing)
if existing:
    files.download(existing[0])  # download the first; repeat for others from file browser
else:
    print('Nothing to download yet. Run a reconstruction cell first.')

Files available: ['frame_0007_1755004067806936064_poisson.ply', 'frame_0007_1755004067806936064_ballpivot.ply', 'frame_0007_1755004067806936064_poisson.stl', 'frame_0007_1755004067806936064_ballpivot.stl', 'frame_0007_1755004067806936064_poisson.obj', 'frame_0007_1755004067806936064_ballpivot.obj', 'frame_0007_1755004067806936064_poisson.glb', 'frame_0007_1755004067806936064_ballpivot.glb']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Notes
- **Normals:** Poisson/Ball-Pivoting need reasonably oriented normals. The notebook estimates normals if missing and orients them consistently; if your cloud is very noisy, try larger `NORMAL_RADIUS`.
- **Downsampling:** Decrease `VOXEL_SIZE` for more detail (longer runtime), increase it if your file is huge.
- **Poisson depth:** Typical range 8–12; higher gives finer detail but costs more.
- **Density trimming:** Removes low-confidence regions. Try 0.01–0.05.
- **Ball-Pivoting radii:** Derived from average NN spacing; tweak `BP_RADII_SCALES`.
- **Decimation:** Set `TARGET_TRIS=0` to keep all triangles.
- **GLB export:** Requires `trimesh` and `pygltflib`. If export fails, you still have `.ply/.stl/.obj`.
